In [4]:
!pip install tensorflow

In [5]:
!pip install opencv-python-headless==4.5.2.52

In [7]:
%tensorflow_version 2.x   # Para garantizar que la versión 2.x sea importada
#print(tf.__version__)
import tensorflow as tf
from google.colab import drive
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, Activation, Flatten, Conv2D, MaxPooling2D
import pickle
import random 
import cv2
import os
import numpy as np

drive.mount('/content/gdrive')
directorio = "gdrive/My Drive/tesis/fotos/carass"
personas = []
carpetas_personas = os.listdir(directorio)

for imagen in carpetas_personas:
    nombre = os.path.splitext(imagen)[0]
    print(nombre)
    personas.append(nombre)
data_entrenamiento = []

print(personas)

def crear_entrenamiento_datos():
        for persona in personas:
            ruta = os.path.join(directorio, persona)
            class_num = personas.index(persona)
            print(class_num)

            for foto in os.listdir(ruta):
                
                try:
                    
                    matriz_foto = cv2.imread(os.path.join(ruta,foto), cv2.IMREAD_GRAYSCALE)
                    matriz_redimensionada = cv2.resize(matriz_foto, (300,300))
                    data_entrenamiento.append([matriz_redimensionada, class_num])
                    
                
                except Exception as e:
                    pass

crear_entrenamiento_datos()

random.shuffle(data_entrenamiento)

x = []
y = []


for caracteristicas, etiqueta in data_entrenamiento:
    x.append(caracteristicas)
    y.append(etiqueta)

x = np.array(x).reshape(-1, 300, 300, 1)
y = np.array(y)

print(x.shape)
print(y.shape)
x = x/255.0

pickle_out = open("nombres.pickle","wb")
pickle.dump(personas, pickle_out)
pickle_out.close()

# esto se usa para tranformar la codificacion de las etiquetas de enteros a one-hot
#con el fin de usar categorical_crossentropy
#y = tf.keras.utils.to_categorical(y, num_classes=3)

`%tensorflow_version` only switches the major version: 1.x or 2.x.
You set: `2.x   # Para garantizar que la versión 2.x sea importada`. This will be interpreted as: `2.x`.


TensorFlow 2.x selected.
Mounted at /content/gdrive
erika
ivan
diego
['erika', 'ivan', 'diego']
0
1
2
(2250, 300, 300, 1)
(2250,)


In [ ]:
modelo = Sequential()

modelo.add(Conv2D(128, (3, 3), input_shape=x.shape[1:], padding="same"))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2, 2)))

modelo.add(Conv2D(128, (3, 3), (1, 1), padding="same"))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2, 2)))

modelo.add(Conv2D(256, (3, 3), (1, 1), padding="same"))
modelo.add(Activation('relu'))
modelo.add(MaxPooling2D(pool_size=(2, 2)))


modelo.add(Flatten())

modelo.add(Dense(128))
modelo.add(Activation('relu'))

modelo.add(Dense(128))
modelo.add(Activation('relu'))

modelo.add(Dense(3))
modelo.add(Activation('softmax'))

modelo.compile(loss='sparse_categorical_crossentropy',
              optimizer='adam',
              metrics=['accuracy'])

modelo.summary()

modelo.fit(x, y, batch_size=10, epochs=5, validation_split=0.4)

#modelo.save('gdrive/My Drive/tesis/modelos/modelo er-iv-di conv2d (128,128,256)(64,128,128) colab 300x300')
#modelo.save('gdrive/My Drive/tesis/modelos/modelo ivan-erika-diego colab 300x300')



Model: "sequential_2"
_________________________________________________________________
 Layer (type)                Output Shape              Param #   
 conv2d_6 (Conv2D)           (None, 300, 300, 128)     1280      
                                                                 
 activation_12 (Activation)  (None, 300, 300, 128)     0         
                                                                 
 max_pooling2d_6 (MaxPooling  (None, 150, 150, 128)    0         
 2D)                                                             
                                                                 
 conv2d_7 (Conv2D)           (None, 150, 150, 128)     147584    
                                                                 
 activation_13 (Activation)  (None, 150, 150, 128)     0         
                                                                 
 max_pooling2d_7 (MaxPooling  (None, 75, 75, 128)      0         
 2D)                                                  

In [ ]:
#convertir el modelo en version de tensorflow lite

converter = tf.lite.TFLiteConverter.from_keras_model(modelo)
#converter = tf.lite.TFLiteConverter.from_saved_model('gdrive/My Drive/tesis/modelos/modelo er-iv-di conv2d (128,128,256)(64,128,128) colab 300x300')
tflite_model = converter.convert()


INFO:tensorflow:Assets written to: /tmp/tmph3klki28/assets


INFO:tensorflow:Assets written to: /tmp/tmph3klki28/assets


In [ ]:
#guardar el modelo en version de tensorflow lite

with open('gdrive/My Drive/tesis/modelos/modelolite er-iv-di conv2d (128,128,256)(64,128,128) colab 300x300', 'wb') as f:
  f.write(tflite_model)